In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
N_NODES = 200
SPECT_RADIUS = 0.9

a = 1
time_scale = np.ones(N_NODES)*a
trainlen = 2200
future = 1000
buffer = 100

In [9]:
import copy

def correct_dimensions(s, targetlength):
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s


def identity(x):
    return x

def step_function(x):
    if x > 0.5:
        return 1
    else:
        return 0
    
def sigmoid(x):
    return 1/(1+np.exp(-10*x+1))


class LI_Deep_ESN:
    def __init__(self, n_inputs, n_outputs, n_reservoir=200, W=None, W_in=None,
                 noise=0.001, input_shift=None, num_layer=None,
                 input_scaling=None, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 out_activation=identity, inverse_out_activation=identity,
                 random_state=None, time_scale=None):
        # check for proper dimensionality of all arguments and write them down.
        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.n_outputs = n_outputs
        self.noise = noise
        self.input_shift = correct_dimensions(input_shift, n_inputs)
        self.input_scaling = correct_dimensions(input_scaling, n_inputs)

        self.teacher_scaling = teacher_scaling
        self.teacher_shift = teacher_shift

        self.out_activation = out_activation
        self.inverse_out_activation = inverse_out_activation
        self.random_state = random_state
        self.time_scale = time_scale
        self.W = W
        self.W_in = W_in
        self.num_layer = num_layer

        # the given random_state might be either an actual RandomState object,
        # a seed or None (in which case we use numpy's builtin RandomState)
        if isinstance(random_state, np.random.RandomState):
            self.random_state_ = random_state
        elif random_state:
            try:
                self.random_state_ = np.random.RandomState(random_state)
            except TypeError as e:
                raise Exception("Invalid seed: " + str(e))
        else:
            self.random_state_ = np.random.mtrand._rand
            
    def _scale_inputs(self, inputs):
        """for each input dimension j: multiplies by the j'th entry in the
        input_scaling argument, then adds the j'th entry of the input_shift
        argument."""
        if self.input_scaling is not None:
            inputs = np.dot(inputs, np.diag(self.input_scaling))
        if self.input_shift is not None:
            inputs = inputs + self.input_shift
        return inputs
    
    def _update(self, state, input_pattern):
        tmp_state = copy.deepcopy(state)
        layer_size = self.W.shape[0]//self.num_layer
        preactivation = np.dot(self.W[:layer_size, :layer_size], state[:layer_size]) + np.dot(self.W_in[:layer_size], input_pattern)
        
        tmp_state[:layer_size] = (1 - self.time_scale[:layer_size]) * state[:layer_size] + self.time_scale[:layer_size] * np.tanh(preactivation)
        
        for i in range(1, self.num_layer-1):
            preactivation = np.dot(self.W[layer_size*i:layer_size*(i+1), layer_size*i:layer_size*(i+1)], state[layer_size*i:layer_size*(i+1)]) \
            + np.dot(self.W[layer_size*(i-1):layer_size*i, layer_size*i:layer_size*(i+1)], state[layer_size*(i-1):layer_size*i])
            
            tmp_state[layer_size*i:layer_size*(i+1)] = (1-self.time_scale[layer_size*i:layer_size*(i+1)])*state[layer_size*i:layer_size*(i+1)] + self.time_scale[layer_size*(i-1):layer_size*i]*np.tanh(preactivation)
        return (tmp_state + self.noise * self.time_scale * (self.random_state_.rand(self.n_reservoir) - 0.5))
    

    def calc_lyapunov_exp(self, inputs, initial_distance, n):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        states1 = np.zeros((inputs.shape[0], self.n_reservoir))
        states2 = np.zeros((inputs.shape[0], self.n_reservoir))
        transient = min(int(inputs.shape[0] / 10), 100)
        for i in range(1, transient):
            states1[i, :] = self._update(states1[i-1], inputs[i, :])
        states2[transient-1, :] = states1[transient-1, :]
        states2[transient-1, n] = states2[transient-1, n] + initial_distance
        gamma_k_list = []
        for k in range(transient, inputs.shape[0]):
            states1[k, :] = self._update(states1[k-1], inputs[k, :])
            states2[k, :] = self._update(states2[k-1], inputs[k, :])
            gamma_k = np.linalg.norm(states2[k, :]-states1[k, :])
            gamma_k_list.append(gamma_k/initial_distance)
            states2[k, :] = states1[k, :] + (initial_distance/gamma_k)*(states2[k, :]-states1[k, :])
        lyapunov_exp = np.mean(np.log(gamma_k_list))
        return lyapunov_exp
            
    
    def fit(self, inputs, outputs):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        if outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))
        inputs_scaled = self._scale_inputs(inputs)
        teachers_scaled = outputs

        # step the reservoir through the given input,output pairs:
        states = np.zeros((inputs.shape[0], self.n_reservoir))
        for n in range(1, inputs.shape[0]):
            states[n, :] = self._update(states[n - 1], inputs_scaled[n, :])
        transient = min(int(inputs.shape[0] / 10), 100)
        extended_states = np.hstack((states, inputs_scaled))
        
        self.W_out = np.dot(np.linalg.pinv(extended_states[transient:, :]),teachers_scaled[transient:, :]).T
        # print(self.W_out.shape)

        # remember the last state for later:
        self.laststate = states[-1, :]
        self.lastinput = inputs[-1, :]
        self.lastoutput = teachers_scaled[-1, :]
            
        # apply learned weights to the collected states:
        pred_train = np.dot(extended_states, self.W_out.T)
        return pred_train

    def predict(self, inputs, continuation=True):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        n_samples = inputs.shape[0]

        if continuation:
            laststate = self.laststate
            lastinput = self.lastinput
            lastoutput = self.lastoutput
        else:
            laststate = np.zeros(self.n_reservoir)
            lastinput = np.zeros(self.n_inputs)
            lastoutput = np.zeros(self.n_outputs)

        inputs = np.vstack([lastinput, inputs])
        states = np.vstack(
            [laststate, np.zeros((n_samples, self.n_reservoir))])
        outputs = np.vstack(
            [lastoutput, np.zeros((n_samples, self.n_outputs))])

        for n in range(n_samples):
            states[n + 1, :] = self._update(states[n, :], inputs[n + 1, :])
            outputs[n + 1, :] = np.dot(self.W_out,np.concatenate([states[n + 1, :], inputs[n + 1, :]]))

        return self.out_activation(outputs[1:])
        # print(outputs[1:])
        # return np.heaviside(outputs[1:]-0.5, 0)*0.3


In [10]:
def make_layered_network(N, average_degree, community_number, mu):
    assert N % community_number == 0, 'N must be devisible by community_number'
    size = N/community_number
    G = np.zeros((N, N))
    for i in range(N):
        com_index = i//size
        k_in_prev = 0
        k_out_prev = 0
        for j in range(int(size*com_index)):
            if G[j][i] != 0:
                k_out_prev += 1
        for j in range(int(size*com_index), int(size*(com_index+1))):
            if G[i][j] != 0:
                k_in_prev += 1
        for j in range(i, N):
            if j < size*((i//size)+1):
                if int(com_index) == 0 or int(com_index) == community_number-1:
                    if np.random.rand() < (average_degree*(1-(mu*0.5))-k_in_prev)/(size-(i-(size*com_index))+1):
                        G[i][j] = np.random.randn()
                        G[j][i] = np.random.randn()
                else:
                    if np.random.rand() < (average_degree*(1-mu)-k_in_prev)/(size-(i-(size*com_index))+1):
                        G[i][j] = np.random.randn()
                        G[j][i] = np.random.randn()
            elif j < size*((i//size)+2):
                if int(com_index) == 0 or int(com_index) == community_number-1:
                    if np.random.rand() < (average_degree*(mu*0.5)-k_out_prev)/size:
                        G[i][j] = np.random.randn()
                        # G[j][i] = np.random.randn()
                else:
                    if np.random.rand() < (average_degree*(mu)-k_out_prev)/size:
                        G[i][j] = np.random.randn()
                        # G[j][i] = np.random.randn()
                    
    return G

In [5]:
def make_data_for_narma(length):
    tau = 0.01
    buffer = 100
    x = np.random.rand(length+100)*0.5
    y = np.zeros(length)
    for i in range(length):
        if i < 29:
            y[i] = 0.2*y[i-1] + 0.004*y[i-1]*np.sum(np.hstack((y[i-29:], y[:i]))) + 1.5*x[i-29+100]*x[i+100] + 0.001
        else:
            y[i] = 0.2*y[i-1] + 0.004*y[i-1]*np.sum(np.hstack((y[i-29:i]))) + 1.5*x[i-29+100]*x[i+100] + 0.001
    return x, y

In [11]:
def calculate_narma(mu, average_degree, num_community):
    narma_list = []
    for k in range(30):
        W = make_layered_network(N_NODES, average_degree, num_community, mu)
        W_IN = (np.random.rand(N_NODES, 1) * 2 - 1)*0.1
        W_IN[int(N_NODES/num_community):] = 0
        radius = np.max(np.abs(np.linalg.eigvals(W)))
        spectral_radius = SPECT_RADIUS
        W = W * (spectral_radius / radius)
        
        data, target = make_data_for_narma(trainlen+future)
        
        esn = LI_Deep_ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=N_NODES,
                          W=W,
                          W_in=W_IN,
                          num_layer=num_community,
                          noise=0,
                          time_scale=time_scale)

        pred_training = esn.fit(data[buffer:trainlen+buffer], target[:trainlen])

        prediction = esn.predict(data[trainlen+buffer:])
        narma_result = np.sqrt(np.mean((np.reshape(prediction, -1)-np.reshape(target[2200:], -1))**2)/np.var(target[2200:]))
        # print(memory_capacity_result)
        narma_list.append(narma_result)
    return np.mean(narma_list), mu

In [12]:
print(calculate_narma(mu=0.1, average_degree=10, num_community=5))

(0.4164702151196699, 0.1)


In [13]:
print(calculate_narma(mu=0.1, average_degree=10, num_community=10))

(0.39795601403021463, 0.1)


In [14]:
print(calculate_narma(mu=0.2, average_degree=10, num_community=10))

(0.40294093300077943, 0.2)


In [15]:
print(calculate_narma(mu=0.3, average_degree=10, num_community=10))

(0.4023969202279271, 0.3)


In [16]:
print(calculate_narma(mu=0.4, average_degree=10, num_community=10))

(0.40288608176791996, 0.4)


In [17]:
print(calculate_narma(mu=0.1, average_degree=20, num_community=10))

(0.40190166830307755, 0.1)


In [18]:
print(calculate_narma(mu=0.1, average_degree=20, num_community=5))

(0.405280659448433, 0.1)


In [19]:
print(calculate_narma(mu=0.08, average_degree=20, num_community=5))

(0.40399802209116964, 0.08)


In [20]:
print(calculate_narma(mu=0.1, average_degree=10, num_community=20))

(0.40781368100370247, 0.1)


In [21]:
print(calculate_narma(mu=0.1, average_degree=20, num_community=1))

(0.42974557169882205, 0.1)


In [22]:
print(calculate_narma(mu=0.1, average_degree=40, num_community=10))

(0.4013313673040777, 0.1)


In [23]:
print(calculate_narma(mu=0.5, average_degree=40, num_community=10))

(0.39850120463906474, 0.5)


In [24]:
print(calculate_narma(mu=0.6, average_degree=20, num_community=10))

(0.4036010347648434, 0.6)


In [25]:
print(calculate_narma(mu=0.5, average_degree=40, num_community=10))

(0.40501308949451253, 0.5)
